<a href="https://colab.research.google.com/github/1900690/Strawberry/blob/main/eye_camera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0.0.1 ファイル作成  
0.0.2 船の動画のＵＲＬ追加  
0.0.3 jsonのリファクタリングサイトのＵＲＬついか  
0.0.4 抽出済みjsonをドライブに追加し,pandasでの整理実験
0.0.5 pandasでデータ抽出、各データの意味を記載  

目標;元々のjsonからeyedataを抽出する方法を考える

テスト用船の動画（15秒）
https://video-ac.com/video/211

In [ ]:
#@title 動画にタイムレコードを描画
from PIL import Image
import cv2

# gifファイル作成用イメージリスト
images =[]

# 動画ファイルのキャプチャー
cap = cv2.VideoCapture('/content/211_640x360.mov')

# 動画ファイルのフレームレート取得
fps = cap.get(cv2.CAP_PROP_FPS)

# 保存用動画ファイルのフォーマット設定
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
out = cv2.VideoWriter('timecode.mp4', fourcc, fps, (320, 180))

# 動画を1コマずつ取り込んで処理
while(cap.isOpened()):
    ret, frame = cap.read() # キャプチャー画像の取り込み

    if ret==True: # キャプチャー画像がある場合
        # 現在時間を取得
        time = cap.get(cv2.CAP_PROP_POS_MSEC)

        # 現在のフレーム数を取得
        frame_n = cap.get(cv2.CAP_PROP_POS_FRAMES)

        # タイムコードを作成
        timecode = str(int(time//1000)).zfill(2)+':'+str(int(frame_n%fps)).zfill(2)

        # 縦横半分にリサイズ
        resize = cv2.resize(frame, (320, 180))

        # タイムコードを動画に追加
        cv2.putText(resize, timecode, (260, 170), cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (0, 0, 255), lineType=cv2.LINE_AA)
        cv2.rectangle(resize, (100, 100), (40, 60), (0, 0, 255))

        # mp4動画を保存
        out.write(resize)

        # opencvからpillow形式に変換しgifファイル作成用リストimagesに追加
        matrix_pil = Image.fromarray(cv2.cvtColor(resize, cv2.COLOR_BGR2RGB))
        images.append(matrix_pil)

    else: # キャプチャー画像がない場合はループ終了
        break

cap.release() # 再生画像をクローズ
out.release() # 出力動画ファイルをクローズ

# gif動画保存
#images[0].save('timecode.gif', save_all=True, append_images=images[1:],
#               optimize=False, duration=1000/fps, loop=0)

In [ ]:
#@title ミリ秒のタイムスタンプを取得
import cv2
#関数定義
def get_timestamps(video_path):
  cap = cv2.VideoCapture(video_path)
  timestamps = [cap.get(cv2.CAP_PROP_POS_MSEC)]
  while cap.isOpened():
    frame_exists, curr_frame = cap.read()
    if not frame_exists:
      break
    timestamp = cap.get(cv2.CAP_PROP_POS_MSEC)
    timestamps.append(timestamp)
  cap.release()
  return timestamps

#動画を指定
movie="/content/timecode.mp4"

#タイムスタンプ取得
timestamps = get_timestamps(movie)
print("タイムスタンプは",timestamps)
print("総フレーム数は",len(timestamps))
#
cap = cv2.VideoCapture(movie)
print("fpsは",cap.get(cv2.CAP_PROP_FPS))
print("動画の長さは",int(len(timestamps)/cap.get(cv2.CAP_PROP_FPS)))

タイムスタンプは [0.0, 0.0, 33.366700033366705, 66.73340006673341, 100.10010010010011, 133.46680013346682, 166.83350016683352, 200.20020020020021, 233.5669002335669, 266.93360026693364, 300.3003003003003, 333.66700033366703, 367.0337003670337, 400.40040040040043, 433.7671004337671, 467.1338004671338, 500.50050050050055, 533.8672005338673, 567.2339005672338, 600.6006006006006, 633.9673006339673, 667.3340006673341, 700.7007007007007, 734.0674007340674, 767.4341007674341, 800.8008008008009, 834.1675008341675, 867.5342008675342, 900.9009009009009, 934.2676009342676, 967.6343009676343, 1001.0010010010011, 1034.3677010343677, 1067.7344010677346, 1101.1011011011012, 1134.4678011344677, 1167.8345011678346, 1201.2012012012012, 1234.5679012345681, 1267.9346012679346, 1301.3013013013012, 1334.6680013346681, 1368.0347013680348, 1401.4014014014015, 1434.7681014347681, 1468.1348014681348, 1501.5015015015017, 1534.8682015348681, 1568.2349015682348, 1601.6016016016017, 1634.9683016349684, 1668.335001668335, 1

#jsonを扱う
jsonをきれいにするには以下のサイトにアクセス
https://jsonformatter.curiousconcept.com/#

※今回扱うデータは大きすぎるため、オンラインのリファクタリングサイトでは1/4づつしかリファクタリングできない

In [1]:
#driveに接続
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#ファイルコピー
import shutil
shutil.copyfile("/content/drive/MyDrive/EyeData.json", "/content/EyeData.json")

'/content/EyeData.json'

参考;  
https://qiita.com/hrsma2i/items/cd089d00165ae16e56a1  
https://note.nkmk.me/python-pandas-json-normalize/

In [21]:
#データを読み込み分解する
import pandas as pd

json_file = "/content/EyeData.json"
df = pd.read_json(json_file)
df2 = pd.json_normalize(df.iloc[0])

df2

,eyeValid,gazeValid,lPupilCenterX,lPupilDiameter,lPupilDiameterMM,pupilRadius0,pupilRadius1,rPupilCenterX,rPupilDiameter,rPupilDiameterMM,...,pupilCenter1.x,pupilCenter1.y,recomPoint.x,recomPoint.y,recomPoint.z,rightExData.blink,rightExData.eyeDataExBitMask,rightExData.eyelidDown,rightExData.eyelidUp,rightExData.openness
0,4,20,-2.543111,0.074114,0.0,0.0,0.0,1.562018,0.070266,0.0,...,0.417595,0.437073,0.375787,0.428088,-1.0,0,1,0.0,0.0,0.0


各データの意味  
このjsonファイルは、EyeDataというキーに対応する値として、目のデータを表すオブジェクトの配列を持っています。各オブジェクトは以下のキーと値を持っています。

**有効なデータの場合**
- eyeValid: 目の有効性を示す数値です。4は両目が有効であることを意味します¹。
- gazePoint0: 左目の視線の座標を示すオブジェクトです。x, y, zはそれぞれ水平方向、垂直方向、奥行き方向の座標を表します。zが-1.0の場合は視線が無効であることを意味します¹。
- gazePoint1: 右目の視線の座標を示すオブジェクトです。x, y, zは同様に座標を表します¹。
- gazeValid: 視線の有効性を示す数値です。20は両目の視線が有効であることを意味します¹。
- gazeVectorLeft: 左目の視線ベクトルを示すオブジェクトです。x, y, zはそれぞれ水平方向、垂直方向、奥行き方向のベクトル成分を表します¹。
- gazeVectorRight: 右目の視線ベクトルを示すオブジェクトです。x, y, zは同様にベクトル成分を表します¹。
- lPupilCenterX: 左目の瞳孔中心の水平方向の座標です²。
- lPupilDiameter: 左目の瞳孔直径です²。
- lPupilDiameterMM: 左目の瞳孔直径（ミリメートル単位）です²。
- leftExData: 左目の拡張データを示すオブジェクトです。blinkはまばたきを示す数値で、0はまばたきしていないことを意味します。eyeDataExBitMaskは拡張データが有効であることを示す数値で、1はeyelidDown, eyelidUp, opennessが有効であることを意味します。eyelidDownは下まぶたの位置、eyelidUpは上まぶたの位置、opennessは目の開き具合を表す数値です³。
- pupilCenter0: 左目の瞳孔中心の座標を示すオブジェクトです。x, yはそれぞれ水平方向、垂直方向の座標を表します。
- pupilCenter1: 右目の瞳孔中心の座標を示すオブジェクトです。x, yは同様に座標を表します。
- pupilRadius0: 左目の瞳孔半径です。
- pupilRadius1: 右目の瞳孔半径です。
- rPupilCenterX: 右目の瞳孔中心の水平方向の座標です²。
- rPupilDiameter: 右目の瞳孔直径です²。
- rPupilDiameterMM: 右目の瞳孔直径（ミリメートル単位）です²。
- recomPoint: 推奨される視点（両目視点）の座標を示すオブジェクトです。x, y, zはそれぞれ水平方向、垂直方向、奥行き方向の座標を表します¹。
- recomValid: 推奨される視点の有効性を示す真偽値です。trueは有効であることを意味します¹。
- rightExData: 右目の拡張データを示すオブジェクトです。blink, eyeDataExBitMask, eyelidDown, eyelidUp, opennessは左目の拡張データと同様に意味します³。
- timestmp: データが取得された時刻を示す数値です¹。
**無効なデータの場合**
- eyeValid: 目の有効性を示す数値です。0は両目が無効であることを意味します。
- gazePoint0: 左目の視線の座標を示すオブジェクトです。x, y, zはそれぞれ水平方向、垂直方向、奥行き方向の座標を表します。zが0.0の場合は視線が無効であることを意味します。
- gazePoint1: 右目の視線の座標を示すオブジェクトです。x, y, zは同様に座標を表します。
- gazeValid: 視線の有効性を示す数値です。0は両目の視線が無効であることを意味します。
- gazeVectorLeft: 左目の視線ベクトルを示すオブジェクトです。x, y, zはそれぞれ水平方向、垂直方向、奥行き方向のベクトル成分を表します。x, y, zが全て0.0の場合は視線ベクトルが無効であることを意味します。
- gazeVectorRight: 右目の視線ベクトルを示すオブジェクトです。x, y, zは同様にベクトル成分を表します。
- lPupilCenterX: 左目の瞳孔中心の水平方向の座標です。
- lPupilDiameter: 左目の瞳孔直径です。
- lPupilDiameterMM: 左目の瞳孔直径（ミリメートル単位）です。
- leftExData: 左目の拡張データを示すオブジェクトです。blinkはまばたきを示す数値で、0はまばたきしていないことを意味します。eyeDataExBitMaskは拡張データが有効であることを示す数値で、1はeyelidDown, eyelidUp, opennessが有効であることを意味します。eyelidDownは下まぶたの位置、eyelidUpは上まぶたの位置、opennessは目の開き具合を表す数値です。
- mpuData: 加速度計、ジャイロスコープ、磁気センサー、四元数データを示すオブジェクトです。accelは加速度計データで、x, y, z軸に沿った加速度（G単位）を表す配列です。gyroはジャイロスコープデータで、x, y, z軸に沿った角速度（度/秒単位）を表す配列です。magは磁気センサーデータで、x, y, z軸に沿った地磁気（μT単位）を表す配列です。quatは四元数データで、姿勢や回転を表す4つの数値からなる配列です。
- pupilCenter0: 左目の瞳孔中心の座標を示すオブジェクトです。x, yはそれぞれ水平方向、垂直方向の座標を表します。
- pupilCenter1: 右目の瞳孔中心の座標を示すオブジェクトです。x, yは同様に座標を表します。
- pupilRadius0: 左目の瞳孔半径です。
- pupilRadius1: 右目の瞳孔半径です。
- rPupilCenterX: 右目の瞳孔中心の水平方向の座標です。
- rPupilDiameter: 右目の瞳孔直径です。
- rPupilDiameterMM: 右目の瞳孔直径（ミリメートル単位）です。
- recomPoint: 推奨される視点（両目視点）の座標を示すオブジェクトです。x, y, zはそれぞれ水平方向、垂直方向、奥行き方向の座標を表します。
- recomValid: 推奨される視点の有効性を示す真偽値です。falseは無効であることを意味します。
- rightExData: 右目の拡張データを示すオブジェクトです。blink, eyeDataExBitMask, eyelidDown, eyelidUp, opennessは左目の拡張データと同様に意味します。
- timestmp: データが取得された時刻を示す数値です。